<a href="https://colab.research.google.com/github/Tien84/Face-Detection/blob/master/GridSearch_ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/driver')

Mounted at /content/driver


In [ ]:
# grid search ARIMA parameters
import warnings
from math import sqrt
from pandas import read_csv
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """
/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import pandas as pd
df=pd.read_csv('/content/driver/My Drive/LSTM_Model/AAPL.csv',header=0, index_col=0, parse_dates=True, squeeze=True)
df=df['Adj Close'].dropna()
print('Shape of data',df.shape)
df.head()
df

Shape of data (2018,)


Date
2012-01-03     56.011238
2012-01-04     56.312252
2012-01-05     56.937424
2012-01-06     57.532646
2012-01-09     57.441391
                 ...    
2020-01-03    293.598297
2020-01-06    295.937744
2020-01-07    294.545959
2020-01-08    299.284088
2020-01-09    305.641113
Name: Adj Close, Length: 2018, dtype: float64

In [ ]:
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(df, arima_order):
	# prepare training dataset
	df = df.astype('float32')
	train_size = int(len(df) * 0.667)
	train, test = df[0:train_size], df[train_size:]
	history = [x for x in train]
	#test.reset_index(drop=True,inplace = True)
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit()
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	rmse = sqrt(mean_squared_error(test, predictions))
	return rmse

In [ ]:
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					rmse = evaluate_arima_model(dataset, order)
					if rmse < best_score:
						best_score, best_cfg = rmse, order
					print('ARIMA%s RMSE=%.3f' % (order,rmse))
				except:
					continue
	print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))

In [ ]:
# load dataset
def parser(x):
	return datetime.strptime('190'+x,'%Y-%m')
# load dataset
df = pd.read_csv('/content/driver/My Drive/LSTM_Model/AAPL.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
# evaluate parameters
'''p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)'''
p_values = [0,1,2]
d_values = [0,1]
q_values = [0,1,2]

warnings.filterwarnings("ignore")
evaluate_models(df['Adj Close'].values, p_values, d_values, q_values)

ARIMA(0, 0, 0) RMSE=83.437
ARIMA(0, 0, 1) RMSE=42.780
ARIMA(0, 1, 0) RMSE=2.987
ARIMA(0, 1, 1) RMSE=2.991
ARIMA(0, 1, 2) RMSE=2.989
ARIMA(1, 1, 0) RMSE=2.990
ARIMA(2, 0, 0) RMSE=40662.220
ARIMA(2, 1, 0) RMSE=2.989
ARIMA(2, 1, 1) RMSE=2.990
Best ARIMA(0, 1, 0) RMSE=2.987
